## Introducción a GANs

### Referencias
- Goodfellow, I., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., Courville, A. and Bengio, Y., 2020. Generative adversarial networks. Communications of the ACM, 63(11), pp.139-144. [link](https://arxiv.org/abs/1406.2661)

- ...

### Que pasos debemos seguir para entrenar una GAN?
- Cargar datos, transformarlos en formato necesario para pytorch. $t_{train}$ va a tener transformaciones diferentes a $t_{test}$, por que? 
- A los datos de entrenamiento debemos dividirlos una vez mas para usar elementos de validación durante el entrenamiento.
- Definir arquitectura de nuestras redes, la red generadora ($G$), evaluadora ($D$) (Ver models.py)
- Definir métodos funciones de error, optimizador y métricas de evaluación para el entrenamiento paralelo de ambas redes.

In [1]:
from torchvision import datasets
from torchvision import transforms as tfs
from torch.utils import data
import PIL

transforms_train = [
    tfs.RandomHorizontalFlip(p=0.7),
    tfs.RandomAffine(0, scale=(0.7, 1.0)),
    tfs.Resize((32, 32)),
    tfs.Grayscale(1),
    tfs.Lambda(lambda x: PIL.ImageOps.invert(x)),
    tfs.ToTensor(),
]

transforms_test = [
    tfs.RandomHorizontalFlip(p=0.7),
    tfs.RandomAffine(0, scale=(0.7, 1.0)),
    tfs.Resize((32, 32)),
    tfs.Grayscale(1),
    tfs.Lambda(lambda x: PIL.ImageOps.invert(x)),
    tfs.ToTensor(),
]

train_data = datasets.ImageFolder(
    "../data/train/", transform=tfs.Compose(transforms_train)
)

test_data = datasets.ImageFolder(
    "../data/test/", transform=tfs.Compose(transforms_test)
)

In [2]:
train_loader = data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=64, shuffle=True)

In [3]:
from models.generator import _G
from models.discriminator import _D
from models.gan import GAN
import PIL.ImageOps

In [4]:
from torchsummary import summary

G = _G(input_h_w=112, latent_v=64)
summary(G, (64,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]          66,560
       BatchNorm1d-2                 [-1, 1024]           2,048
              ReLU-3                 [-1, 1024]               0
            Linear-4               [-1, 100352]     102,860,800
       BatchNorm1d-5               [-1, 100352]         200,704
              ReLU-6               [-1, 100352]               0
   ConvTranspose2d-7           [-1, 64, 56, 56]         131,136
       BatchNorm2d-8           [-1, 64, 56, 56]             128
              ReLU-9           [-1, 64, 56, 56]               0
  ConvTranspose2d-10          [-1, 1, 112, 112]           1,025
          Sigmoid-11          [-1, 1, 112, 112]               0
Total params: 103,262,401
Trainable params: 103,262,401
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00


In [5]:
D = _D(input_h_w=112)
summary(D, (1, 112, 112))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           1,088
         LeakyReLU-2           [-1, 64, 56, 56]               0
            Conv2d-3          [-1, 128, 28, 28]         131,200
       BatchNorm2d-4          [-1, 128, 28, 28]             256
         LeakyReLU-5          [-1, 128, 28, 28]               0
            Linear-6                 [-1, 1024]     102,761,472
       BatchNorm1d-7                 [-1, 1024]           2,048
         LeakyReLU-8                 [-1, 1024]               0
            Linear-9                    [-1, 1]           1,025
          Sigmoid-10                    [-1, 1]               0
Total params: 102,897,089
Trainable params: 102,897,089
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 5.38
Params size (MB): 392.52
E

In [7]:
gan = GAN(epochs=100, input_h_w=32, data_loader=train_loader, gpu_mode=False)
gan.train()
print("Training finished!")

training start!!
Avg one epoch time: 8.44, total 100 epochs time: 847.68
Training finish!... save training results
Training finished!


### Ejercicios
- 